<img src="schema.jpg" width="800"/>


In [3]:
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install tqdm
!pip3 install regex
!pip3 install ipython

In [2]:
import pandas as pd

file_path = "./code/dataset/raw_dataset.csv"
raw_dataset = pd.read_csv(file_path)
# print("Raw dataset")
print(raw_dataset[:1])

       id                                  change_id   
0  367660  I69b6e153324a3e5680e096cd714e5d4dd05bae34  \

                                 message_id  author  real_author   
0  7989310a7c88de651a3367a1c0c831369639eff9    2394         2394  \

                      created_at                message  revision_number   
0  2019-09-04 20:13:52.000000000  Uploaded patch set 1.                1  \

   is_bot     username  
0   False  adam.spiers  


In [3]:
import pandas as pd


def get_second(time_str):
    segment = time_str.split(" ")
    second = 0
    print("Time", time_str)
    for i in segment:
        if i[-1] == "s":
            second += int(i[:-1])
        elif i[-1] == "m":
            second += int(i[:-1]) * 60
        elif i[-1] == "h":
            second += int(i[:-1]) * 60 * 60
        elif i[-1] == "d":
            second += int(i[:-1]) * 60 * 60 * 24
        else:
            pass
            # print(time_str)
    return second


df = pd.read_csv(file_path, parse_dates=["created_at"])
df["is_bot"] = df["is_bot"].astype(bool)
# print(df.dtypes)

In [1]:
# 1.) Created dataset that start with build failed CI then attempt recheck -> still failure or change to success
from tqdm import tqdm
import re
from IPython.display import display

tqdm.pandas()
rerun_builds_info = dict()
df = raw_dataset.copy()[:1000]
# (df.groupby(["change_id", "revision_number"]))["message"].apply(display)
df.sort_values(by=["created_at"], inplace=True)

users_rechecks = {}

for key, group in tqdm(df.groupby(["change_id", "revision_number"])):
    # Skip groups with length 1, because there is no recheck
    if len(group) == 1:
        continue
    change_id, revision_number = key
    build, recheck, build_then_recheck = False, False, False
    final_status = dict()
    prev_status = dict()
    computation_time = 0

    total_max_time = 0
    max_time = 0

    dict_bot = {}
    recheck_requester = []
    rnd = 0

    start_time = ""
    end_time = ""
    recheck_count = 0
    print(change_id)
    for ind, col in group.iterrows():
        # start with build failed CI
        # start loop if build fail is found
        time = col["created_at"]

        if not start_time:
            start_time = time

        if not end_time or time > end_time:
            end_time = time

        if not build and col["is_bot"] and "build" in col["message"].lower():
            build = True

        if not build:
            continue

        if col["is_bot"]:
            try:
                patch_fail = False
                build = re.findall(
                    r"((?:merge|build) (?:succeeded|failed))",
                    col["message"],
                    re.IGNORECASE,
                )
                bot = re.findall(
                    r"((?:success|failure|aborted) in(?: \w+?)*s)",
                    col["message"],
                    re.IGNORECASE,
                )

                if any(
                    ("fail" in b.lower() or "abort" in b.lower()) for b in build
                ):  # build maybe replaced by bot:
                    patch_fail = True

                if build and recheck:
                    # Count computation_time for bot
                    bot_time = 0
                    for i in bot:
                        bot_time += get_second(i.split(" in ")[1])

                        if max_time < get_second(i.split(" in ")[1]):
                            max_time = get_second(i.split(" in ")[1])

                    computation_time += bot_time
            except:
                pass

            # process bot info,
            if rnd not in dict_bot:
                dict_bot[rnd] = dict()

            if col["username"] not in dict_bot[rnd]:
                dict_bot[rnd][col["username"]] = {"status": "", "jobs": []}

            # Compute jobs parameters
            bot_jobs = re.findall(
                r"-(.*?(success|failure|aborted) in(?: \w+?)*s *(?:\(non-voting\))?)",
                col["message"],
                re.IGNORECASE,
            )

            for bot_job in bot_jobs:
                job = bot_job[0]
                job = job.strip().replace(": ", "")
                is_voting = not job.endswith("(non-voting)")
                job = job.replace(" (non-voting)", "")
                job_components = job.split(" ", 2)
                job_name = job_components[0]
                job_link = job_components[1]
                job_status_and_time = job_components[2]
                print(bot_job)
                print(job_components)
                job_status, job_time = job_status_and_time.split(" in ")
                job_time = get_second(job_time)
                dict_bot[rnd][col["username"]]["jobs"].append(
                    {
                        "name": job_name,
                        "link": job_link,
                        "voting": is_voting,
                        "status": job_status.lower(),
                        "time_in_seconds": job_time,
                    }
                )

            # Calculate status
            if dict_bot[rnd][col["username"]]["status"] == "failure":
                final_status[col["username"]] = "failure"
            else:
                if patch_fail:
                    dict_bot[rnd][col["username"]]["status"] = "failure"
                else:
                    dict_bot[rnd][col["username"]]["status"] = "success"

                final_status[col["username"]] = "failure" if patch_fail else "success"

        # Check if recheck
        if "recheck" in col["message"].lower() and col["is_bot"] == False:
            # print(col["username"],patch_fail)
            recheck = True
            rnd += 1
            recheck_requester.append(
                {
                    f"{rnd}": {
                        "user": col["username"],
                        "build_status": "failed" if patch_fail else "success",
                        "created_at": col["created_at"],
                    }
                }
            )
            if col["username"] not in users_rechecks:
                users_rechecks[f'{col["username"]}'] = []
            users_rechecks[f'{col["username"]}'].append(
                {
                    "build_status": "failed" if patch_fail else "success",
                    "created_at": col["created_at"],
                }
            )
            if len(final_status) > 0:
                prev_status = final_status
            final_status = dict()

            total_max_time += max_time
            max_time = 0

            if build:
                build_then_recheck = True

            recheck_count += 1

    total_max_time += max_time
    if build_then_recheck:
        status = ""

        if len(final_status) == 0:
            final_status = prev_status

        for username in final_status:
            if status == "failure":
                continue
            if final_status[username] == "failure":
                status = "failure"
            if final_status[username] == "":
                status = ""
            if final_status[username] == "success":
                status = "success"

        # recheck_count depends on human
        # human should write recheck so it should recheck
        # Status is for patch
        rerun_builds_info[key] = {
            "requestors": recheck_requester,
            "computation_time": computation_time,
            "status": status,
            "bot": dict_bot,
            "waiting_time": total_max_time,
            "code_review_time": get_second(end_time) - get_second(start_time),
            "recheck_count": recheck_count,
        }

# {('Ife8387a3bdb75717b896cf1067d2490084665e49', 1): {'computation_time': 105799,
#   'status': 'failure',
#   'bot': {0: {'powerkvm': 'failure',
#     'openstack-ironic-ci': 'failure',
#     'zuul': 'failure'},
#    1: {'powerkvm': 'failure',
#     'zuul': 'success',
#     'openstack-ironic-ci': 'failure'}},
#   'waiting_time': 8791,
#   'code_review_time': 0,
#   'recheck_count': 1},

# {
#     (change_id, revision_number): {
#         computation_time,
#         status,
#         "bot": dict_bot: {
#           rerun_number: {name_of_bot: status}
#         },
#         "waiting_time": total_max_time,
#         "code_review_time": get_second(end_time) - get_second(start_time),
#         recheck_count,
#     }
# }


display(rerun_builds_info)
# print(users_rechecks)

NameError: name 'raw_dataset' is not defined

In [14]:
# extract features [time, text, keyword, test name (functional, system etc.)]
# for each bot?
# rerun_number: {name_of_bot:
# [{name_of_test, status_of_specific_test, computation_time_of_specific_test, voting | non-voting}]
# }
# row 1 [good to run, long time] | label success_rerun
# row 2 [                     ]  | label failed_rerun


# output - dictionary

# DONE
# change_id revision_number
# pick the build with change_id and revision_number
# filter by is bot
# Collect test names, their time, status, voting ot non-voting


# get full info for rerun_builds
# rerun_builds_info_keys = list(rerun_builds_info.keys())
# rerun_builds_info_keys_dictionary = [
#     {"change_id": change_id, "revision_number": revision_number}
#     for change_id, revision_number in rerun_builds_info_keys
# ]

# rerun_builds_full_info = df.merge(
#     pd.DataFrame.from_dict(rerun_builds_info_keys_dictionary),
#     on=["change_id", "revision_number"],
#     how="inner",
# )
# print(rerun_builds_full_info)

        id                                  change_id   
0   367722  Ife8387a3bdb75717b896cf1067d2490084665e49  \
1   367723  Ife8387a3bdb75717b896cf1067d2490084665e49   
2   367724  Ife8387a3bdb75717b896cf1067d2490084665e49   
3   367725  Ife8387a3bdb75717b896cf1067d2490084665e49   
4   367726  Ife8387a3bdb75717b896cf1067d2490084665e49   
5   367727  Ife8387a3bdb75717b896cf1067d2490084665e49   
6   367728  Ife8387a3bdb75717b896cf1067d2490084665e49   
7   367729  Ife8387a3bdb75717b896cf1067d2490084665e49   
8   367730  Ife8387a3bdb75717b896cf1067d2490084665e49   
9   367731  Ife8387a3bdb75717b896cf1067d2490084665e49   
10  367732  Ife8387a3bdb75717b896cf1067d2490084665e49   
11  367733  Ife8387a3bdb75717b896cf1067d2490084665e49   
12  367734  Ife8387a3bdb75717b896cf1067d2490084665e49   
13  367735  Ife8387a3bdb75717b896cf1067d2490084665e49   
14  367736  Ife8387a3bdb75717b896cf1067d2490084665e49   
15  367737  Ife8387a3bdb75717b896cf1067d2490084665e49   
16  367738  Ife8387a3bdb75717b8

In [ ]:
# run the logistic? regression model

In [2]:
# work on recheck

# 1. Build information
# 2. Author name, comment should contain upload patch, create a list of people who call recheck and the end status of build
# 3. How many recheck of the users has success
# 4. Patch code information, files, directories

# Start on build features (job name, time, voting non-voting)

# Patch information
# file level or directory level

# from patch extract which directory was changes
# changes to the specific
# dependency of code parts

# Based on person (some persons recheck fails more than other)
# Tendency metrics

# Tendency of the job which are failing

# Tendency metrics should be recalculated on each recheck

# Tendency of particular tests
# authors, who send recheck


# Dont care about comments
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [ ]:
# Check if they remain non-voting

#  empty is 0